In [1]:
!pip install haystack-ai
!pip install "datasets>=2.6.1"
!pip install "sentence-transformers>=3.0.0"

  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl (664.8 MB)
Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl (127.9 MB)
  Attempting uninstall: nvidia-cusolver-cu12
    Found existing installation: nvidia-cusolver-cu12 11.6.3.83
    Uninstalling nvidia-cusolver-cu12-11.6.3.83:
      Successfully uninstalled nvidia-cusolver-cu12-11.6.3.83


In [2]:
from haystack.telemetry import tutorial_running

tutorial_running(35)

In [3]:
from datasets import load_dataset
from haystack import Document

dataset = load_dataset("vblagoje/PubMedQA_instruction", split="train")
dataset = dataset.select(range(1000))
all_documents = [Document(content=doc["context"]) for doc in dataset]
all_questions = [doc["instruction"] for doc in dataset]
all_ground_truth_answers = [doc["response"] for doc in dataset]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
from typing import List
from haystack import Pipeline
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.writers import DocumentWriter
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.document_stores.types import DuplicatePolicy

document_store = InMemoryDocumentStore()

document_embedder = SentenceTransformersDocumentEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
document_writer = DocumentWriter(document_store=document_store, policy=DuplicatePolicy.SKIP)

indexing = Pipeline()
indexing.add_component(instance=document_embedder, name="document_embedder")
indexing.add_component(instance=document_writer, name="document_writer")

indexing.connect("document_embedder.documents", "document_writer.documents")

indexing.run({"document_embedder": {"documents": all_documents}})

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

{'document_writer': {'documents_written': 1000}}

In [5]:
import os
from getpass import getpass
from haystack.components.builders import AnswerBuilder, ChatPromptBuilder
from haystack.dataclasses import ChatMessage
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI API key:")

template = [
    ChatMessage.from_user(
        """
        You have to answer the following question based on the given context information only.

        Context:
        {% for document in documents %}
            {{ document.content }}
        {% endfor %}

        Question: {{question}}
        Answer:
        """
    )
]

rag_pipeline = Pipeline()
rag_pipeline.add_component(
    "query_embedder", SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
)
rag_pipeline.add_component("retriever", InMemoryEmbeddingRetriever(document_store, top_k=3))
rag_pipeline.add_component("prompt_builder", ChatPromptBuilder(template=template))
rag_pipeline.add_component("generator", OpenAIChatGenerator(model="o1-mini-2024-09-12"))
rag_pipeline.add_component("answer_builder", AnswerBuilder())

rag_pipeline.connect("query_embedder", "retriever.query_embedding")
rag_pipeline.connect("retriever", "prompt_builder.documents")
rag_pipeline.connect("prompt_builder.prompt", "generator.messages")
rag_pipeline.connect("generator.replies", "answer_builder.replies")
rag_pipeline.connect("retriever", "answer_builder.documents")

Enter OpenAI API key:··········


🚅 Components
  - query_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: ChatPromptBuilder
  - generator: OpenAIChatGenerator
  - answer_builder: AnswerBuilder
🛤️ Connections
  - query_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - retriever.documents -> answer_builder.documents (List[Document])
  - prompt_builder.prompt -> generator.messages (List[ChatMessage])
  - generator.replies -> answer_builder.replies (List[ChatMessage])

In [6]:
question = "Do high levels of procalcitonin in the early phase after pediatric liver transplantation indicate poor postoperative outcome?"

response = rag_pipeline.run(
    {
        "query_embedder": {"text": question},
        "prompt_builder": {"question": question},
        "answer_builder": {"query": question},
    }
)
print(response["answer_builder"]["answers"][0].data)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Yes, high levels of procalcitonin (PCT) in the early postoperative phase after pediatric liver transplantation are indicative of poorer postoperative outcomes. In the observed study, pediatric patients with elevated PCT levels on postoperative day 2 experienced several adverse outcomes, including:

- **Higher International Normalized Ratio (INR) Values on Day 5:** Elevated INR suggests impaired liver function and coagulation abnormalities.
- **Increased Incidence of Primary Graft Non-Function:** Patients were more likely to experience failure of the transplanted liver to function properly.
- **Longer Stay in the Pediatric Intensive Care Unit (ICU):** Higher PCT levels were associated with extended ICU hospitalization.
- **Prolonged Duration on Mechanical Ventilation:** Patients required longer periods of respiratory support.

Importantly, the elevated PCT levels were not correlated with systemic infections but were associated with other indicators of liver stress and dysfunction, such 

In [7]:
import random

questions, ground_truth_answers, ground_truth_docs = zip(
    *random.sample(list(zip(all_questions, all_ground_truth_answers, all_documents)), 25)
)

In [8]:
rag_answers = []
retrieved_docs = []

for question in list(questions):
    response = rag_pipeline.run(
        {
            "query_embedder": {"text": question},
            "prompt_builder": {"question": question},
            "answer_builder": {"query": question},
        }
    )
    print(f"Question: {question}")
    print("Answer from pipeline:")
    print(response["answer_builder"]["answers"][0].data)
    print("\n-----------------------------------\n")

    rag_answers.append(response["answer_builder"]["answers"][0].data)
    retrieved_docs.append(response["answer_builder"]["answers"][0].documents)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Question: Do plasma levels of Galectin-9 reflect disease severity in malaria infection?
Answer from pipeline:
Yes, plasma levels of Galectin-9 reflect disease severity in malaria infection. In the study involving 50 acute malaria cases from Thailand, patients with severe malaria (SM) exhibited significantly higher Galectin-9 levels compared to those with uncomplicated malaria (UM) at both day 0 (923 vs. 617 pg/mL, P = 0.03) and day 7 (659 vs. 348 pg/mL, P = 0.02). Additionally, higher Galectin-9 levels were associated with indicators of more severe disease, such as a blood urea nitrogen to creatinine ratio (BUN/creatinine) ≥20 mg/dL. These findings indicate that elevated plasma Galectin-9 levels are correlated with increased disease severity in malaria infections.

-----------------------------------



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Question: Does ginsenoside Rg3 improve erectile function in streptozotocin-induced diabetic rats?
Answer from pipeline:
Yes, ginsenoside Rg3 improves erectile function in streptozotocin-induced diabetic rats. In the study described, diabetic rats treated with 100 mg/kg of Rg3 showed a significantly higher intracavernosal pressure (ICP) relative to mean arterial pressure compared to control groups. Additionally, Rg3 treatment tended to normalize the expression of key proteins related to apoptosis and vascular function, increased the number of positively stained nerve fibers, and reduced the apoptotic index in corpus cavernosum cells. These findings indicate that Rg3 has a protective effect on erectile function in this diabetic rat model.

-----------------------------------



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Question: Does beta-blocker Treatment Worsen Critical Limb Ischemia in Patients Receiving Endovascular Therapy?
Answer from pipeline:
No, beta-blocker treatment does not worsen critical limb ischemia (CLI) in patients receiving endovascular therapy (EVT). The study found no significant differences in amputation-free survival (AFS), limb salvage rates, overall survival, or freedom from major adverse limb events (MALE) between CLI patients treated with beta-blockers and those who were not. Specifically, at three years, AFS was 58.8% for beta-blocker-treated patients compared to 58.5% for non-treated patients (log-rank p = 0.76), and similar non-significant differences were observed for limb salvage rate, overall survival, and freedom from MALE. Therefore, beta-blocker therapy does not appear to worsen clinical outcomes in CLI patients undergoing EVT.

-----------------------------------



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Question: Does recipient T cell TIM-3 and hepatocyte galectin-9 signalling protect mouse liver transplants against ischemia-reperfusion injury?
Answer from pipeline:
Yes, recipient T cell TIM-3 and hepatocyte galectin-9 (Gal-9) signaling protect mouse liver transplants against ischemia-reperfusion injury (IRI). 

**Evidence from the Context:**

1. **TIM-3 Transgenic Models:**
   - **Protection Against IRI:** Orthotopic liver transplants in groups where both donors and recipients were TIM-3 transgenic (TIM-3Tg→TIM-3Tg) exhibited resistance to IR-stress. This was demonstrated by preserved hepatocellular function (as indicated by serum ALT levels) and intact liver architecture (Suzuki's score).
   - **Susceptibility in Non-Tg Recipients:** In contrast, when only the donor was TIM-3 transgenic but the recipient was wild type (WT or TIM-3Tg→WT), the liver transplants were susceptible to IRI.

2. **Mechanisms of Protection:**
   - **Modulation of T Cell Responses:** TIM-3 induction in recipi

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Question: Are secretory phospholipases A2 secreted from ciliated cells and increase mucin and eicosanoid secretion from goblet cells?
Answer from pipeline:
Yes, secretory phospholipases A₂ (sPLA₂) are secreted from ciliated-enriched airway epithelial cells and subsequently increase both mucin and eicosanoid (specifically cysteinyl leukotrienes) secretion from goblet-enriched cells. The study found that:

- **sPLA₂ Expression and Secretion:** sPLA₂ groups IIa, V, and X showed increased mRNA expression in ciliated-enriched cells but not in goblet-enriched cells. Moreover, sPLA₂ proteins were secreted from the apical side of ciliated-enriched cells and were strongly present in these cells based on immunostaining.

- **Effect on Goblet Cells:** The secreted sPLA₂, particularly sPLA₂ V, enhanced the secretion of cysteinyl leukotrienes (cysLTs) from goblet-enriched cells. Additionally, sPLA₂ V was shown to increase mucin secretion from goblet cells. This mucin secretion was inhibited by lipo

While each evaluator is a component that can be run individually in Haystack, they can also be added into a pipeline. This way, we can construct an `eval_pipeline` that includes all evaluators for the metrics we want to evaluate our pipeline on.

In [9]:
from haystack.components.evaluators.document_mrr import DocumentMRREvaluator
from haystack.components.evaluators.faithfulness import FaithfulnessEvaluator
from haystack.components.evaluators.sas_evaluator import SASEvaluator

eval_pipeline = Pipeline()
eval_pipeline.add_component("doc_mrr_evaluator", DocumentMRREvaluator())
eval_pipeline.add_component("faithfulness", FaithfulnessEvaluator())
eval_pipeline.add_component("sas_evaluator", SASEvaluator(model="sentence-transformers/all-MiniLM-L6-v2"))

results = eval_pipeline.run(
    {
        "doc_mrr_evaluator": {
            "ground_truth_documents": list([d] for d in ground_truth_docs),
            "retrieved_documents": retrieved_docs,
        },
        "faithfulness": {
            "questions": list(questions),
            "contexts": list([d.content] for d in ground_truth_docs),
            "predicted_answers": rag_answers,
        },
        "sas_evaluator": {"predicted_answers": rag_answers, "ground_truth_answers": list(ground_truth_answers)},
    }
)

100%|██████████| 25/25 [01:32<00:00,  3.72s/it]


### Constructing an Evaluation Report

Once we've run our evaluation pipeline, we can also create a full evaluation report. Haystack provides an `EvaluationRunResult` which we can use to display an `aggregated_report` 👇

In [10]:
from haystack.evaluation.eval_run_result import EvaluationRunResult

inputs = {
    "question": list(questions),
    "contexts": list([d.content] for d in ground_truth_docs),
    "answer": list(ground_truth_answers),
    "predicted_answer": rag_answers,
}

evaluation_result = EvaluationRunResult(run_name="pubmed_rag_pipeline", inputs=inputs, results=results)
evaluation_result.aggregated_report()

{'metrics': ['doc_mrr_evaluator', 'faithfulness', 'sas_evaluator'],
 'score': [0.98, np.float64(1.0), np.float64(0.7095784056186676)]}

#### Extra: You can also see a detailed report with the scores for each sample in your dataset, and we will choose the output format as DataFrame

In [11]:
results_df = evaluation_result.detailed_report(output_format='df')
results_df

,question,contexts,answer,predicted_answer,doc_mrr_evaluator,faithfulness,sas_evaluator
0,Do plasma levels of Galectin-9 reflect disease...,[Galectin-9 (Gal-9) is a β-galactoside-binding...,"Gal-9 is released during acute malaria, and re...","Yes, plasma levels of Galectin-9 reflect disea...",1.0,1.0,0.549785
1,Does ginsenoside Rg3 improve erectile function...,[Ginsenoside Rg3 is one of the active ingredie...,Oral gavage with Rg3 appears to both prevent d...,"Yes, ginsenoside Rg3 improves erectile functio...",1.0,1.0,0.549989
2,Does dual trigger for final oocyte maturation ...,[To identify the risk factors for suboptimal r...,Basal LH level was useful predictor of the sub...,"Yes, administering a dual trigger (gonadotropi...",0.5,1.0,0.777647
3,Are high levels of serum vitamin D associated ...,[There are conflicting results for relationshi...,High levels of serum vitamin D were associated...,"Yes, according to the provided context, high l...",1.0,1.0,0.826247
4,Does periodontitis promote the diabetic develo...,[Emerging evidence has indicated the bad effec...,This study provided new evidence for the posit...,"Yes, based on the provided context, periodonti...",1.0,1.0,0.949997
5,Does over-expression of small ubiquitin-like m...,[Non-small cell lung cancer (NSCLC) is one of ...,SENP1 may be a promising predictor of survival...,"Yes, the over-expression of small ubiquitin-li...",1.0,1.0,0.744036
6,Do whole-genome sequencing and epidemiological...,[Mycobacterium abscessus has emerged as a majo...,We have not demonstrated cross-transmission of...,"Yes, whole-genome sequencing combined with epi...",1.0,1.0,0.618438
7,Is community readiness for adolescents ' overw...,[South Africa is undergoing epidemiological an...,Religious leaders recognised that they act as ...,"Yes, the community readiness for preventing ov...",1.0,1.0,0.771028
8,Does low perfusion index affect the difference...,"[In emergency cases, finger stick testing is p...","In the emergency department, perfusion index v...","Yes, a low perfusion index is associated with ...",1.0,1.0,0.572686
9,Does biolimus-eluting stent with biodegradable...,[To investigate clinical outcomes of coronary ...,"BES, compared with SES, significantly improved...","Yes, the use of a biolimus-eluting stent (BES)...",1.0,1.0,0.494062


Having our evaluation results as a dataframe can be quite useful. For example, below we can use the pandas dataframe to filter the results to the top 3 best scores for semantic answer similarity (`sas_evaluator`) as well as the bottom 3 👇


In [12]:
import pandas as pd

top_3 = results_df.nlargest(3, "sas_evaluator")
bottom_3 = results_df.nsmallest(3, "sas_evaluator")
pd.concat([top_3, bottom_3])

,question,contexts,answer,predicted_answer,doc_mrr_evaluator,faithfulness,sas_evaluator
4,Does periodontitis promote the diabetic develo...,[Emerging evidence has indicated the bad effec...,This study provided new evidence for the posit...,"Yes, based on the provided context, periodonti...",1.0,1.0,0.949997
15,Does puerarin inhibit the inflammatory respons...,[The isoflavone puerarin [7-hydroxy-3-(4-hydro...,This study indicates that the effect of puerar...,"Yes, puerarin inhibits the inflammatory respon...",1.0,1.0,0.885431
16,Does mediastinal transposition of the omentum ...,[The greater omentum has been found to be immu...,Mediastinal transposition of the omentum decre...,"Yes, mediastinal transposition of the omentum ...",1.0,1.0,0.862128
12,Does geographical disparity in breast reconstr...,[Breast reconstruction (BR) following mastecto...,Geographical barriers to accessing BR have red...,"Yes, the geographical disparity in breast reco...",1.0,1.0,0.255245
9,Does biolimus-eluting stent with biodegradable...,[To investigate clinical outcomes of coronary ...,"BES, compared with SES, significantly improved...","Yes, the use of a biolimus-eluting stent (BES)...",1.0,1.0,0.494062
0,Do plasma levels of Galectin-9 reflect disease...,[Galectin-9 (Gal-9) is a β-galactoside-binding...,"Gal-9 is released during acute malaria, and re...","Yes, plasma levels of Galectin-9 reflect disea...",1.0,1.0,0.549785


## What's next

🎉 Congratulations! You've learned how to evaluate a RAG pipeline with model-based evaluation frameworks and without any labeling efforts.

If you liked this tutorial, you may also enjoy:
- [Serializing Haystack Pipelines](https://haystack.deepset.ai/tutorials/29_serializing_pipelines)
-  [Creating Your First QA Pipeline with Retrieval-Augmentation](https://haystack.deepset.ai/tutorials/27_first_rag_pipeline)

To stay up to date on the latest Haystack developments, you can [sign up for our newsletter](https://landing.deepset.ai/haystack-community-updates). Thanks for reading!